In [1]:
!pip3 install pandas transformers

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.0 MB 2.1 MB/s eta 0:00:05
   ---- ----------------------------------- 1.0/10.0 MB 2.5 MB/s eta 0:00:04
   -------- ------------------------------- 2.1/10.0 MB 3.4 MB/s eta 0:00:03
   ---------------- ----------------------- 4.2/10.0 MB 4.9 MB/s eta 0:00:02
   ---------------- ----------------------- 4.2/10.0 MB 4.9 MB/s eta 0:00:02
   -------------------- ------------------- 5.2/10.0 MB 4.5 MB/s eta 0:00:02
   ---------------------------- ----------- 7.1/10.0 MB 4.6 MB/s eta 0:00:01
   ------------------------------------- -- 9.4/10.0 MB 5.4 MB/s eta 0:00:01
   ---------------------------------------- 10.0/10.0 MB 5.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ----------------


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import re
from transformers import AutoTokenizer

# Install dependencies
!pip3 install openpyxl urduhack

from urduhack.normalization import normalize
from urduhack.stop_words import STOP_WORDS


C:\Users\Talha\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


In [ ]:

# Load data
file_path = 'news_article_combined.xlsx'  
try:
    data = pd.read_excel(file_path)
except FileNotFoundError:
    print(f"File {file_path} not found.")
    exit()
except Exception as e:
    print(f"An error occurred: {e}")
    exit()

# Data cleaning
data = data.drop_duplicates()
data = data.dropna(subset=['title', 'content', 'gold_label'])
data['gold_label'] = data['gold_label'].replace({'science-technology-technology': 'science-technology'})

def clean_text(text):
    text = normalize(text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in STOP_WORDS]
    return ' '.join(filtered_words)

data['content'] = data['content'].apply(clean_text).apply(remove_stopwords)
data['title'] = data['title'].apply(clean_text).apply(remove_stopwords)

# Save cleaned data
cleaned_file_path = 'cleaned_news_articles.csv'
data.to_csv(cleaned_file_path, index=False)
print(f"Cleaned data saved to {cleaned_file_path}")

try:
    tokenizer = AutoTokenizer.from_pretrained("hadidev/gpt2-urdu-tokenizer-withgpt2")
except Exception as e:
    print(f"Error loading tokenizer: {e}")
    exit()

data['tokenized_content'] = data['content'].apply(
    lambda x: tokenizer.encode(x, truncation=True, max_length=512)
)
data['token_length'] = data['tokenized_content'].apply(len)

tokenized_file_path = 'tokenized_news_articles.csv'
data.to_csv(tokenized_file_path, index=False)
print(f"Tokenized data saved to {tokenized_file_path}")

print(data[['content', 'tokenized_content', 'token_length']].head())


Cleaned data saved to cleaned_news_articles.csv
Tokenized data saved to tokenized_news_articles.csv
                                             content  \
0  لاہور دوستی انٹرنیشنل تھیٹر فیسٹیول آغاز ، شان...   
1  بالی ووڈ دبنگ خان ممبئی موجود شاہ رخ خان پرتعی...   
2  شہرت بلندیوں چھونے بالی ووڈ اداکار پربھاس بڑا ...   
3  آرٹس کونسل آف پاکستان کراچی زیر اہتمام 21 روزہ...   
4  پاکستان شوبز اداکار ہدایتکار یاسر نواز سوشل می...   

                                   tokenized_content  token_length  
0  [6891, 3671, 5845, 10261, 16898, 749, 105, 83,...           248  
1  [15745, 12008, 26196, 700, 4365, 562, 906, 232...           146  
2  [37833, 14900, 15949, 7231, 12008, 6436, 34512...           169  
3  [76499, 3520, 1123, 415, 1141, 1108, 4919, 211...           512  
4  [1757, 36909, 6436, 23172, 7380, 1477, 5701, 1...           133  
